In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls '/content/drive/MyDrive/mami_data'

# import zipfile
 
# zip_file = "/content/drive/MyDrive/mami_data/training.zip"
 
# try:
#     with zipfile.ZipFile(zip_file) as z:
#         z.extractall("/content/drive/MyDrive/mami_data/training")
#         print("Extracted all")
# except:
#     print("Invalid file")

!ls '/content/drive/MyDrive/mami_data/training/TRAINING/training.csv'

# import zipfile
 
# zip_file = "/content/drive/MyDrive/mami_data/trial.zip"
 
# try:
#     with zipfile.ZipFile(zip_file) as z:
#         z.setpassword(pwd = bytes('*MaMiSemEval2022!', 'utf-8'))
#         z.extractall("/content/drive/MyDrive/mami_data/trial")
#         print("Extracted all")
# except:
#     print("Invalid file")

!ls '/content/drive/MyDrive/mami_data/trial/Users/fersiniel/Desktop/MAMI - TO LABEL/TRIAL DATASET'

#import zipfile
 
#zip_file = "/content/drive/MyDrive/mami_data/test.zip"
 
#try:
#   with zipfile.ZipFile(zip_file) as z:
#       z.setpassword(pwd = bytes('*MaMiSemEval2022!', 'utf-8'))
#        z.extractall("/content/drive/MyDrive/mami_data/test")
#        print("Extracted all")
#except:
#    print("Invalid file")

!ls '/content/drive/MyDrive/mami_data/test/test/Test.csv'

#!cp '/content/drive/MyDrive/mami_data/trial/Users/fersiniel/Desktop/MAMI - TO LABEL/TEST DATASET/test.csv' '/content/drive/MyDrive/mami_data/test'

!cp '/content/drive/MyDrive/mami_data/trial/Users/fersiniel/Desktop/MAMI - TO LABEL/TRIAL DATASET/trial.csv' '/content/drive/MyDrive/mami_data/trial'

applying preprocessing with cleaningg tokenizing and lematization

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import seaborn as sns
data1=pd.read_csv('/content/drive/MyDrive/mami_data/training/TRAINING/training.csv', sep='\t')
data2=pd.read_csv("/content/drive/MyDrive/mami_data/trial/trial.csv",sep='\t')
#training_label=data['misogynous']
# print(training_data.head())
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words=stopwords.words('english')
stop_words.append('imgflipcom')
stop_words.append('zip')
print(stop_words)
lemmatizer=WordNetLemmatizer()
#training data
for index,row in data1.iterrows():
  #print(row)
  filter_sentence=[]
  sentence=row['Text Transcription']
  sentence = sentence.lower()
  #print(sentence)
  sentence=re.sub(r'[^\w\s]','',sentence)#cleaning
  words=nltk.word_tokenize(sentence)
  words=[w for w in words if not w in stop_words]
  for word in words:
    filter_sentence.append(lemmatizer.lemmatize(word))
  #print(filter_sentence)
  listToStr = ' '.join([str(elem) for elem in filter_sentence])
  data1.loc[index,"Text Transcription"]=listToStr
#trail data
for index,row in data2.iterrows():
  #print(row)
    filter_sentence=[]
    sentence=row['Text Transcription']
    sentence = sentence.lower()
    #print(sentence)
    sentence=re.sub(r'[^\w\s]','',sentence)#cleaning
    words=nltk.word_tokenize(sentence)
    words=[w for w in words if not w in stop_words]
    for word in words:
        filter_sentence.append(lemmatizer.lemmatize(word))
    #print(filter_sentence)
    listToStr = ' '.join([str(elem) for elem in filter_sentence])
    data2.loc[index,"Text Transcription"]=listToStr
print(data1.head())
print(data1.shape)
print(data2.head())
print(data2.shape)
#data=pd.concat([data1,data2])
#print(data.head())
#print(data.shape)

applying countvectorizer

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
training_data=data1['Text Transcription']
training_label=data1['misogynous']
X_train_counts = count_vect.fit_transform(training_data)
X_train_counts.shape


TF-IDF transormer

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
print(X_train_tfidf)


from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings("ignore")
clf = MultinomialNB()
clf.fit(X_train_tfidf,training_label)
docs_new = ["kill him because he raped her"]
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
print(predicted)


from sklearn.pipeline import Pipeline
import numpy as np
from sklearn import metrics
from sklearn.model_selection import cross_val_score
sns.countplot(training_label)
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB(alpha=1)),
 ])
scores=cross_val_score(text_clf,training_data,training_label,cv=3,scoring='accuracy')
print(scores)
print(scores.mean())


To find best parameter 

tuning hyper parameter using grid search cv

from sklearn.model_selection import GridSearchCV
param_grid={'alpha': [0,0.5,1]}
print(param_grid)
clf=MultinomialNB()
grid=GridSearchCV(clf,param_grid,cv=3,scoring='accuracy',n_jobs=-1)
grid.fit(X_train_tfidf,training_label)
#grid.grid_scores_
grid.cv_results_
means = grid.cv_results_['mean_test_score']
params = grid.cv_results_['params']
grid.mean_scores=means
print(grid.mean_scores)
#plt.plot(k_range,grid.mean_scores)
#plt.xlabel('Value of k for KNN')
#plt.ylabel('Cross Validated accuracy')
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

#validation accuracy for training data
clf=MultinomialNB(alpha=1)
scores=cross_val_score(clf,X_train_tfidf,training_label,cv=3,scoring='accuracy')
print(scores)
print(scores.mean())

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
y_train=data2['Text Transcription']
y_test=data2['misogynous']
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf',MultinomialNB(alpha=1))])
text_clf.fit(training_data,training_label)
y_pred = text_clf.predict(y_train)
accuracy_score(y_test,y_pred)
metrics.confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test,y_pred
     ))

text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB(alpha=1))])
text_clf.fit(training_data, training_label)
test_data=pd.read_csv("/content/drive/MyDrive/mami_data/test/test/Test.csv",sep='\t')
testing_data=test_data['Text Transcription']
predictednew = text_clf.predict(testing_data)
print(predictednew)

from google.colab import drive
drive.mount('/content/drive')

#!unzip /content/drive/MyDrive/mami_data/train_pixel1.zip


#!ls /content/drive/MyDrive/mami_data/training/TRAINING/

#!unzip -P *MaMiSemEval2022! /content/drive/MyDrive/mami_data/training/TRAINING/trial.zip


# !unzip -P *MaMiSemEval2022! /content/drive/MyDrive/mami_data/training/TRAINING/test.zip

first creating csv file with feature extraction from images and reading csv file

#import glob
import pandas as pd
#import numpy as np
from keras.preprocessing.image import img_to_array,array_to_img,load_img
train_image = pd.read_csv('train_pixel1.csv')
print(train_image.shape)
test_image = pd.read_csv('/content/drive/MyDrive/mami_data/test_pixel.csv')
print(test_image.shape)


#reading training and testing datasets
import pandas as pd
total_data_train=pd.read_csv('/content/drive/MyDrive/mami_data/training/TRAINING/training.csv',sep='\t')
total_data_test=pd.read_csv('/content/drive/MyDrive/mami_data/trial/trial.csv',sep='\t')
print(total_data_train['file_name'])

#y_train is extracting from training dataset by checking filename and file_list
import os
file_list=os.listdir(r"/content/drive/MyDrive/mami_data/training/TRAINING/training2_data/training/")
#print(file_list)
y_train=[]
for j in range(0,2001): 
  for i in range(len(total_data_train)):
      if(file_list[j]==(total_data_train['file_name'][i])):
        y_train.append(total_data_train['misogynous'][i])
      else:
        continue
print(len(y_train))

#y_test is extracting from training dataset by checking filename and file_list
file_list2=os.listdir(r"/content/drive/MyDrive/mami_data/trial/Users/fersiniel/Desktop/MAMI - TO LABEL/TRIAL DATASET/")
y_test=[]
for j in file_list2: 
  for i in range(len(total_data_test)):
      if(j==(total_data_test['file_name'][i])):
        y_test.append(total_data_test['misogynous'][i])
      else:
        continue

X_train=train_image#train_image is a feature extraction from image dataset
X_test=test_image#train_image is a feature extraction from image dataset

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)
print(len(X_train))

Naive bayes algorithm is applied to model

import numpy as np
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB( )
from sklearn import metrics
from sklearn.model_selection import cross_val_score
sns.countplot(y_train)
clf=MultinomialNB( )
scores=cross_val_score(clf,X_train,y_train,cv=3,scoring='accuracy')
print(scores)
print(len(X_train))
print(scores.mean())


To find best parameter 

import matplotlib.pyplot as plt


tuning hyper parameter using grid search cv

from sklearn.model_selection import GridSearchCV
param_grid=param_grid={'alpha': [0,0.5,1]}
print(param_grid)
clf=MultinomialNB( )
grid=GridSearchCV(clf,param_grid,cv=3,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
#grid.grid_scores_
grid.cv_results_
means = grid.cv_results_['mean_test_score']
params = grid.cv_results_['params']
#for mean,param in zip(means,params):
#print("%f  with:   %r" % (mean,param))
grid.mean_scores=means
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

clf=MultinomialNB(alpha=0)
scores=cross_val_score(clf,X_train,y_train,cv=3,scoring='accuracy')
print(scores)
print(scores.mean())


clf= MultinomialNB(alpha=0)
clf.fit(X_train, y_train)
y_pred_image = clf.predict(X_test)
print(metrics.classification_report(y_test,y_pred_image))



Mounted at /content/drive
